In [1]:
! python -m pip install --no-index --find-links=../input/openvino-wheels -r ../input/openvino-wheels/requirements.txt

Looking in links: ../input/openvino-wheels
Processing /kaggle/input/openvino-wheels/openvino_dev-2024.6.0-17404-py3-none-any.whl (from openvino-dev[onnx]==2024.6.0->-r ../input/openvino-wheels/requirements.txt (line 1))
Processing /kaggle/input/openvino-wheels/networkx-3.1-py3-none-any.whl (from openvino-dev==2024.6.0->openvino-dev[onnx]==2024.6.0->-r ../input/openvino-wheels/requirements.txt (line 1))
Processing /kaggle/input/openvino-wheels/openvino_telemetry-2025.1.0-py3-none-any.whl (from openvino-dev==2024.6.0->openvino-dev[onnx]==2024.6.0->-r ../input/openvino-wheels/requirements.txt (line 1))
Processing /kaggle/input/openvino-wheels/openvino-2024.6.0-17404-cp311-cp311-manylinux2014_x86_64.whl (from openvino-dev==2024.6.0->openvino-dev[onnx]==2024.6.0->-r ../input/openvino-wheels/requirements.txt (line 1))
Processing /kaggle/input/openvino-wheels/fastjsonschema-2.17.1-py3-none-any.whl (from openvino-dev[onnx]==2024.6.0->-r ../input/openvino-wheels/requirements.txt (line 1))
  Att

In [2]:
import os
import gc
import warnings
import logging
import time
import math
import cv2
from pathlib import Path
import joblib

import numpy as np
import pandas as pd
import librosa
import soundfile as sf
from soundfile import SoundFile 
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.cuda.amp import autocast, GradScaler # 尽管此处未使用，但保留原始依赖
import timm
from tqdm.auto import tqdm # 进度条，此处未使用
from glob import glob
import torchaudio
import random
import itertools
from typing import Union

# OpenVINO 特有导入
from openvino.tools import mo # 用于模型转换 (较旧API，推荐使用 ov.convert_model)
import openvino as ov
from openvino.runtime import Core # 用于模型加载和推理
import openvino.torch # 对于 PyTorch 前端至关重要

warnings.filterwarnings("ignore") # 忽略警告
logging.basicConfig(level=logging.ERROR) # 设置日志级别为ERROR，忽略普通信息

class CFG:
    """
    全局配置类
    """
    seed = 42 # 随机种子
    print_freq = 100 # 打印频率
    num_workers = 4 # 数据加载工作线程数

    stage = 'train_bce' # 模型阶段 (此处为训练阶段的名称，转换时实际不影响)

    train_datadir = '/kaggle/input/birdclef-2025/train_audio' # 训练音频目录
    train_csv = '/kaggle/input/birdclef-2025/train.csv' # 训练CSV文件
    test_soundscapes = '/kaggle/input/birdclef-2025/test_soundscapes' # 测试声景目录
    submission_csv = '/kaggle/input/birdclef-2025/sample_submission.csv' # 提交文件模板
    taxonomy_csv = '/kaggle/input/birdclef-2025/taxonomy.csv' # 分类学CSV文件
    model_files = ['/kaggle/input/bird2025-sed-ckpt/sedmodel.pth'] # 模型文件路径列表
 
    model_name = 'seresnext26t_32x4d' # 模型骨干网络名称 
    pretrained = False # 是否使用预训练权重 (此处为False，实际加载的是本地检查点)
    in_channels = 1 # 输入通道数 (音频频谱图为单通道)

    SR = 32000 # 采样率
    target_duration = 5 # 目标输出片段时长 (例如，每5秒一个预测)
    train_duration = 10 # 模型训练时使用的音频片段时长 (例如，模型期望10秒的输入)
    
    device = 'cpu' # 推理设备 (此处设置为CPU)

cfg = CFG()
print(f"使用设备: {cfg.device}")
print(f"加载分类数据...")
taxonomy_df = pd.read_csv(cfg.taxonomy_csv)
species_ids = taxonomy_df['primary_label'].tolist()
num_classes = len(species_ids)
print(f"类别数量: {num_classes}")

def set_seed(seed=42):
    """
    设置随机种子以确保结果可复现
    """
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed) # CUDA随机种子，即使不使用CUDA也安全
    torch.cuda.manual_seed_all(seed) # 所有CUDA设备随机种子
    torch.backends.cudnn.deterministic = True # 确保确定性行为
    torch.backends.cudnn.benchmark = False # 禁用CuDNN基准测试以提高确定性

set_seed(cfg.seed)

class AttBlockV2(nn.Module):
    """
    注意力模块 V2
    """
    def __init__(self, in_features: int, out_features: int, activation="linear"):
        super().__init__()

        self.activation = activation # 激活函数类型
        self.att = nn.Conv1d( # 注意力卷积层
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True,
        )
        self.cla = nn.Conv1d( # 分类卷积层
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True,
        )

        self.init_weights() # 初始化权重

    def init_weights(self):
        """初始化卷积层权重"""
        init_layer(self.att)
        init_layer(self.cla)

    def forward(self, x):
        # x: (批次大小, 输入特征维度, 时间步长)
        norm_att = torch.softmax(torch.tanh(self.att(x)), dim=-1) # 注意力权重，经过tanh和softmax
        cla = self.nonlinear_transform(self.cla(x)) # 分类输出，经过非线性变换
        x = torch.sum(norm_att * cla, dim=2) # 将注意力权重应用于分类输出并求和
        return x, norm_att, cla # 返回片段级输出、注意力权重和帧级分类输出

    def nonlinear_transform(self, x):
        """根据配置应用非线性变换"""
        if self.activation == "linear":
            return x
        elif self.activation == "sigmoid":
            return torch.sigmoid(x)


def init_layer(layer):
    """初始化神经网络层的权重"""
    nn.init.xavier_uniform_(layer.weight) # 使用Xavier均匀分布初始化

    if hasattr(layer, "bias"):
        if layer.bias is not None:
            layer.bias.data.fill_(0.0) # 偏差初始化为0

def init_bn(bn):
    """初始化BatchNorm层的权重和偏差"""
    bn.bias.data.fill_(0.0)
    bn.weight.data.fill_(1.0)



class BirdCLEFModel(nn.Module):
    """
    BirdCLEF 竞赛主模型，用于声音事件检测
    """
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg # 加载自检查点的模型配置字典
        
        # 类别数量，从cfg字典中获取，如果不存在则使用全局num_classes
        self.num_classes = cfg.get('num_classes', num_classes) 

        self.bn0 = nn.BatchNorm2d(self.cfg['n_mels']) # Mel频谱图的初始BatchNorm层
        
        self.backbone = timm.create_model( # 创建骨干网络模型
            self.cfg['model_name'], # 模型名称，例如 'seresnext26t_32x4d'
            pretrained=False, # 导出时通常不使用预训练，因为权重已加载
            in_chans=self.cfg['in_channels'], # 输入通道数
            drop_rate=0.0, # 导出时禁用Dropout以获得稳定的计算图
            drop_path_rate=0.0, # 导出时禁用DropPath以获得稳定的计算图
        )

        layers = list(self.backbone.children())[:-2] # 获取骨干网络除最后两层外的所有层
        self.encoder = nn.Sequential(*layers) # 构建编码器
        
        # 根据骨干网络名称确定其输出特征维度
        if "efficientnet" in self.cfg['model_name']:
            backbone_out = self.backbone.classifier.in_features
        elif "eca" in self.cfg['model_name']:
            backbone_out = self.backbone.head.fc.in_features
        elif "res" in self.cfg['model_name']: # 包含 seresnext 系列
            backbone_out = self.backbone.fc.in_features
        else:
            backbone_out = self.backbone.num_features
            
        self.fc1 = nn.Linear(backbone_out, backbone_out, bias=True) # 全连接层
        self.att_block = AttBlockV2(backbone_out, self.num_classes, activation="sigmoid") # 注意力模块
        self.melspec_transform = torchaudio.transforms.MelSpectrogram(
            sample_rate=self.cfg['SR'],
            hop_length=self.cfg['hop_length'],
            n_mels=self.cfg['n_mels'],
            f_min=self.cfg['f_min'],
            f_max=self.cfg['f_max'],
            n_fft=self.cfg['n_fft'],
            pad_mode="constant",
            norm="slaney",
            onesided=True,
            mel_scale="htk",
        )

        self.melspec_transform = self.melspec_transform.cpu()

        self.db_transform = torchaudio.transforms.AmplitudeToDB(
            stype="power", top_db=80
        )
    def extract_feature(self, x):
        """
        从频谱图提取特征
        x 的形状为 (批次大小, 1, Mel通道数, 帧数)
        """
        x = x.permute((0, 1, 3, 2)) # 调整维度顺序 (B, 1, Freq, Time) -> (B, 1, Time, Freq)
        # frames_num = x.shape[2] # 帧数，此处不直接用于图导出

        x = x.transpose(1, 3) # 为 BatchNorm2d 准备 (B, Freq, Time, 1) -> (B, Time, 1, Freq) 维度不对
        # 更正：BatchNorm2d 期望 (N, C, H, W)。如果将 (B, 1, Time, Freq) 视为 (N, C, H, W)，则 H=Time, W=Freq。
        # 那么 C 应该是 1。
        # 如果 self.bn0 是 nn.BatchNorm2d(cfg['n_mels'])，那么它期望 C = n_mels。
        # 因此，x 的形状应该是 (B, n_mels, Time, 1) 或者 (B, n_mels, 1, Time)
        # 原始代码的 bn0 应用在 (B, C, H, W) 形状，其中 C=n_mels。
        # x = x.transpose(1, 3) 这一步将 (B, 1, Time, Freq) 变成 (B, Freq, Time, 1)。
        # 也就是 (B, n_mels, Time, 1)。这与 BatchNorm2d(n_mels) 的期望一致。
        x = self.bn0(x) # 应用批量归一化
        x = x.transpose(1, 3) # 再次调整维度顺序 (B, n_mels, Time, 1) -> (B, 1, Time, n_mels)
        
        x = x.transpose(2, 3) # 调整为 (B, 1, n_mels, Time) 适应骨干网络输入
        
        # (批次大小, 通道数, 频率, 帧数)
        x = self.encoder(x) # 通过编码器（骨干网络）提取特征
        
        # (批次大小, 通道数, 帧数)
        x = torch.mean(x, dim=2) # 在频率维度上进行平均池化
        
        # 通道平滑
        x1 = F.max_pool1d(x, kernel_size=3, stride=1, padding=1)
        x2 = F.avg_pool1d(x, kernel_size=3, stride=1, padding=1)
        x = x1 + x2
        
        # Dropout 在 model.eval() 模式下会自动跳过，导出时不会出现在图中
        x = F.dropout(x, p=0.5, training=self.training)
        x = x.transpose(1, 2) # (批次大小, 帧数, 通道数)
        x = F.relu_(self.fc1(x)) # 激活全连接层
        x = x.transpose(1, 2) # (批次大小, 通道数, 帧数)
        x = F.dropout(x, p=0.5, training=self.training)
        return x, x.shape[-1] # 返回特征图和其时间维度的大小
        

    def forward(self, x):
        """
        模型的前向传播，用于 OpenVINO 导出
        x 是原始音频输入，形状为 (批次大小, 1, 采样点数)
        """

        x_features, _ = self.extract_feature(x) # 从频谱图提取特征

        # AttBlockV2 在 activation="sigmoid" 时，其输出 (clipwise_output 和 segmentwise_output) 已经是 sigmoid 概率
        (clipwise_output, norm_att, segmentwise_output) = self.att_block(x_features)

        # 为了方便在 Python/NumPy 中进行 TTA 后处理，我们返回帧级别的概率和片段级别的概率。
        # segmentwise_output 默认形状为 (batch_size, num_classes, time_frames)。
        # 此处将其转置为 (batch_size, time_frames, num_classes) 以方便后续处理。
        # clipwise_output 形状为 (batch_size, num_classes)。
        
        # 原始模型的 forward 返回 torch.logit(clipwise_output)。
        # 为了保持一致性，我们在这里也返回 logit。
        return torch.logit(clipwise_output), segmentwise_output.transpose(1, 2) # 返回片段级 logits 和帧级概率


# --- OpenVINO 模型转换函数 ---
def convert_pytorch_to_openvino(pytorch_model, model_name, output_dir, example_input_shape):
    """
    将 PyTorch 模型转换为 OpenVINO IR (Intermediate Representation) 格式。

    Args:
        pytorch_model: 一个已训练的 PyTorch 模型实例。
        model_name (str): 目标 OpenVINO 模型文件的名称 (例如 'my_model')。
        output_dir (str): 保存 .xml 和 .bin 文件的目录。
        example_input_shape (tuple): 模型期望的输入形状 (例如 (批次大小, 通道数, 音频采样点数))。
    Returns:
        str: 转换后的 OpenVINO IR 模型的 .xml 文件路径，如果转换失败则返回 None。
    """
    os.makedirs(output_dir, exist_ok=True) # 创建输出目录
    onnx_path = os.path.join(output_dir, f"{model_name}.onnx") # ONNX 模型路径
    xml_path = os.path.join(output_dir, f"{model_name}.xml") # OpenVINO IR 模型路径
    
    pytorch_model.eval() # 设置模型为评估模式 (禁用 Dropout 等)
    pytorch_model.cpu() # 将模型移动到 CPU 进行导出 (通常导出在CPU上进行)

    print(f"将 PyTorch 模型 '{model_name}' 导出为 ONNX...")
    dummy_input = torch.randn(example_input_shape) # 创建虚拟输入以追踪模型计算图
    
    try:
        torch.onnx.export(pytorch_model,
                          dummy_input,
                          onnx_path,
                          verbose=False, # 不打印详细的 ONNX 图信息
                          opset_version=17, # 推荐的 ONNX opset 版本，兼容性好
                          input_names=['input_audio'], # 为输入层定义名称
                          output_names=['clipwise_logits', 'segmentwise_probabilities'], # 为输出层定义名称
                          dynamic_axes={'input_audio': {0: 'batch_size'}} # 允许批次大小是动态的
                         )
        print(f"ONNX 模型已保存到: {onnx_path}")
    except Exception as e:
        print(f"导出 ONNX 失败: {e}")
        return None

    # 2. 使用 Model Optimizer (ov.convert_model) 将 ONNX 模型转换为 OpenVINO IR 格式
    print(f"使用 Model Optimizer 将 ONNX 模型转换为 OpenVINO IR...")
    try:
        # ov.convert_model 是 OpenVINO 最新版本推荐的转换 API
        ov_model = ov.convert_model(onnx_path, 
                                   )
        
        # 保存 OpenVINO IR 模型 (.xml 和 .bin 文件)
        ov.save_model(ov_model, xml_path)
        print(f"OpenVINO IR 模型已保存到: {xml_path} 和 {Path(xml_path).with_suffix('.bin')}")
        return xml_path
    except Exception as e:
        print(f"转换为 OpenVINO IR 失败: {e}")
        return None

def load_openvino_model(xml_path, device="CPU"):
    """
    加载 OpenVINO IR 模型并编译。

    Args:
        xml_path (str): OpenVINO IR 模型的 .xml 文件路径。
        device (str): 推理设备 (例如 "CPU", "GPU", "NPU")。
    Returns:
        openvino.runtime.CompiledModel: 编译后的 OpenVINO 模型对象。
    """
    core = Core() # 创建 OpenVINO Core 对象
    model = core.read_model(model=xml_path) # 读取 OpenVINO IR 模型
    
    # 编译模型以优化到指定设备
    compiled_model = core.compile_model(model=model, device_name=device)
    print(f"OpenVINO 模型 '{Path(xml_path).stem}' 已编译到设备: {device}")
    return compiled_model

# --- 主执行块 (模型转换和示例推理) ---
if __name__ == "__main__":
    
    # 加载检查点以获取训练时使用的配置 (例如 n_mels, SR, hop_length 等)
    checkpoint = torch.load(cfg.model_files[0], map_location='cpu', weights_only=False)
    cfg_temp = checkpoint["cfg"] # 这包含了训练时的所有必要参数

  
    input_shape_for_export = (1, cfg.in_channels,256, 768 )
    print(f"ONNX 导出使用的输入形状: {input_shape_for_export}")

    # 使用从检查点加载的配置实例化模型
    dummy_model = BirdCLEFModel(cfg_temp) 
    dummy_model.load_state_dict(checkpoint['model_state_dict']) # 加载训练好的权重

    # 将 PyTorch 模型转换为 OpenVINO IR
    output_ir_path = convert_pytorch_to_openvino(
        dummy_model, 
        cfg.model_name, 
        output_dir="./openvino_models", 
        example_input_shape=input_shape_for_export
    )

    if output_ir_path:
        print(f"\n模型转换成功，IR 路径: {output_ir_path}")
        
        # --- 测试 OpenVINO 模型推理 ---
        print("\n--- 测试 OpenVINO 模型推理 ---")
        ov_compiled_model = load_openvino_model(output_ir_path, device="CPU")

        # OpenVINO 推理的输入必须是 NumPy 数组，且形状与导出时一致
        batch_size_for_test = 4 # 模拟单段推理
        test_audio_input = np.random.rand(batch_size_for_test, cfg.in_channels, 256, 768).astype(np.float32)
        print(f"OpenVINO 推理使用的示例输入形状: {test_audio_input.shape}")
        
        # 执行推理
        # 通过导出时定义的输出名称来访问结果
        results = ov_compiled_model(test_audio_input)
        
        # 结果是字典，键是 output_names 中定义的名称
        clipwise_logits = results['clipwise_logits'] # 片段级 logits
        segmentwise_probabilities = results['segmentwise_probabilities'] # 帧级概率

        print(f"OpenVINO 推理片段级 logits 形状: {clipwise_logits.shape}")
        print(f"OpenVINO 推理帧级概率形状: {segmentwise_probabilities.shape}")

    else:
        print("模型转换失败。")


使用设备: cpu
加载分类数据...
类别数量: 206
ONNX 导出使用的输入形状: (1, 1, 256, 768)
将 PyTorch 模型 'seresnext26t_32x4d' 导出为 ONNX...
ONNX 模型已保存到: ./openvino_models/seresnext26t_32x4d.onnx
使用 Model Optimizer 将 ONNX 模型转换为 OpenVINO IR...
OpenVINO IR 模型已保存到: ./openvino_models/seresnext26t_32x4d.xml 和 openvino_models/seresnext26t_32x4d.bin

模型转换成功，IR 路径: ./openvino_models/seresnext26t_32x4d.xml

--- 测试 OpenVINO 模型推理 ---
OpenVINO 模型 'seresnext26t_32x4d' 已编译到设备: CPU
OpenVINO 推理使用的示例输入形状: (4, 1, 256, 768)
OpenVINO 推理片段级 logits 形状: (4, 206)
OpenVINO 推理帧级概率形状: (4, 24, 206)
